In [37]:
import pandas as pd
import os
import s3fs

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

BUCKET = "cgadeau"
FILE_KEY_S3 = "Antoine Airdrop/df_all_clusters_final.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    dataset = pd.read_csv(file_in, sep=",")

In [38]:
dataset

,ID_ABONNE,DATE_ACTE_REEL,DATE_DEMARRAGE_PROMO,TYPE_PROMON,STATUT_FIN_M_MOINS_1,DATE_PRISE_EFFET,DATE_FIN_ABO_PREC,DATE_FIN_REABO,REGION,FORMULE_PREC,...,MOYEN_PAIEMENT,REABO_AVANT_ECHEANCE,DELAI_REABO,NB_APPARITIONS,MOY_DELAI,PREMIERE_APPARITION,ANCIENNETE,NOMBRE_ABONNEMENTS,SCORE_FIDELITE,Cluster_8
0,51276418001,2022-06-19,2022-06-19,ODD 30 jours TC,1.0,2022-06-19,2022-05-31,2022-09-18,Sénégal - Divers,AFRIQUE - EVASION & CANAL+,...,Cash,0,19,9,17.111111,2021-02-16,986,"[5, 4]",0.346154,4
1,51276418001,2021-12-01,2021-12-01,ODD 21 jours TC,0.0,2021-12-01,2021-08-23,2022-02-28,Sénégal - Divers,AFRIQUE - EVASION,...,Cash,0,100,9,17.111111,2021-02-16,986,"[5, 4]",0.346154,4
2,51276418001,2021-05-15,2021-05-15,ODD 15 jours TC,1.0,2021-05-15,2021-04-30,2021-06-14,Dakar,AFRIQUE - EVASION & CANAL+,...,Cash,0,15,9,17.111111,2021-02-16,986,"[5, 4]",0.346154,4
3,51276418001,2022-01-16,2022-01-16,ODD 15 jours TC,1.0,2022-03-01,2022-02-28,2022-05-31,Sénégal - Divers,AFRIQUE - EVASION & CANAL+,...,Cash,1,-43,9,17.111111,2021-02-16,986,"[5, 4]",0.346154,4
4,51276418001,2021-07-24,2021-07-24,ODD 15 jours TC,0.0,2021-07-24,2021-06-14,2021-08-23,Dakar,AFRIQUE - EVASION & CANAL+,...,Cash,0,40,9,17.111111,2021-02-16,986,"[5, 4]",0.346154,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946857,51458332904,2021-02-09,2021-02-09,PAS_ODD,1.0,2021-02-09,2021-02-07,2022-02-08,Dakar,AFRIQUE - EVASION & CANAL+,...,Cash,0,2,3,3.666667,2021-02-09,993,"[1, 1, 1]",0.076923,4
946858,51458332904,2023-02-16,2023-02-16,PAS_ODD,1.0,2023-02-16,2023-02-14,2024-02-15,Dakar,AFRIQUE - EVASION & CANAL+,...,Cash,0,2,3,3.666667,2021-02-09,993,"[1, 1, 1]",0.076923,4
946859,51720983201,2021-09-11,2021-10-01,PAS_ODD,1.0,2021-10-01,2021-09-30,2022-09-30,Dakar,AFRIQUE - ACCESS,...,Prélèvement RIB,1,-19,3,-19.000000,2021-09-11,779,"[1, 1, 1]",0.076923,4
946860,51720983201,2022-09-11,2022-10-01,PAS_ODD,1.0,2022-10-01,2022-09-30,2023-09-30,Dakar,AFRIQUE - ACCESS,...,Prélèvement RIB,1,-19,3,-19.000000,2021-09-11,779,"[1, 1, 1]",0.076923,4


In [46]:
def date_bascule(df, cluster):
    data = df.copy()
    data = data.loc[data["Cluster_8"] == cluster]
    data['DATE_PRISE_EFFET'] = pd.to_datetime(data['DATE_PRISE_EFFET'])
    data['ANNEE_MOIS'] = data['DATE_PRISE_EFFET'].dt.strftime('%Y-%m')
    result = data.groupby('ANNEE_MOIS')['ID_ABONNE'].nunique().reset_index()
    result.rename(columns={'ID_ABONNE' : 'NOMBRE_ABONNEMENTS'}, inplace = True)

    def ecart_type_avant_après(dataframe, line):
        result_before = dataframe.iloc[line:]
        result_after = dataframe.iloc[:line]
        std_before = result_before['NOMBRE_ABONNEMENTS'].std()
        mean_before = result_before['NOMBRE_ABONNEMENTS'].mean()
        std_after = result_after['NOMBRE_ABONNEMENTS'].std()
        mean_after = result_after['NOMBRE_ABONNEMENTS'].mean()
        return (std_before, std_after, mean_before, mean_after)

    min_std = float('inf')
    date = None
    min_std_before = None
    min_std_after = None
    the_mean_before = None
    the_mean_after = None
    for line in range(1, result.shape[0]):
        std_before, std_after, mean_before, mean_after = ecart_type_avant_après(result, line)
        std_line = std_before + std_after
        if min_std > std_line:
            min_std = std_line
            min_std_before = std_before
            min_std_after = std_after
            the_mean_before = mean_before
            the_mean_after = mean_after
            date = line

    mois_bascule = result.loc[date, 'ANNEE_MOIS']
    print('Date = ', mois_bascule)
    print('Std error before = ', min_std_before)
    print('Std error after = ', min_std_after)
    print('Mean before = ', the_mean_before)
    print('Mean after = ', the_mean_after)
    print('Ecart des moyennes = ')

    return(result, mois_bascule)

liste_bascule = []
for i in range (0, 8, 1):
    print(i)
    liste_bascule += [date_bascule(dataset, i)[1]]
print(liste_bascule)

0
Date =  2021-09
Std error before =  1314.2953566894257
Std error after =  38.18376618407357
Mean before =  2388.730769230769
Mean after =  29.0
Ecart des moyennes = 
1
Date =  2022-07
Std error before =  419.1377607660756
Std error after =  772.4214789714683
Mean before =  1311.0625
Mean after =  3624.6666666666665
Ecart des moyennes = 
2
Date =  2021-07
Std error before =  2442.5056857932336
Std error after =  1716.4049250298328
Mean before =  4346.892857142857
Mean after =  3240.3333333333335
Ecart des moyennes = 
3
Date =  2021-05
Std error before =  1470.3906103911427
Std error after =  742.9762221408345
Mean before =  6352.933333333333
Mean after =  4773.5
Ecart des moyennes = 
4
Date =  2023-04
Std error before =  343.11992905438433
Std error after =  1470.2171041493611
Mean before =  1050.5714285714287
Mean after =  2456.222222222222
Ecart des moyennes = 
5
Date =  2021-12
Std error before =  1998.8416260048725
Std error after =  69.37518768743381
Mean before =  3958.130434782